<h1 style="color:pink;">📊 Excel Data ETL Script</h1>
<h3 style="color:purple;">This script involves: </h3>

        🔍 Detecting & Handling Excel File Formats
        🔐 Decrypting Encrypted Files
        📐 Analyzing Workbook Structures
        🔄 Consolidating Data into DataFrames

=========================================================================================================================

# Code Viewing Settings

In [ ]:
import pandas as pd
from pathlib import Path
import msoffcrypto
import tempfile
import csv

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
#pd.set_option('display.width', None)
#np.set_printoptions(threshold=np.inf)
#pd.set_option('display.float_format', lambda x: '%.5f' % x)

=========================================================================================================================

# Read excel
- with multiple workbook 
- with multiple worksheets 
- with consistent column format

In [ ]:
def is_encrypted(file_path):
    try:
        with open(file_path, "rb") as f:
            office_file = msoffcrypto.OfficeFile(f)
            return office_file.is_encrypted()
    except Exception as e:
        print(f"❌ Encryption check failed: {file_path} – {e}")
        return False

def decrypt_excel(file_path, password):
    try:
        with open(file_path, "rb") as f:
            office_file = msoffcrypto.OfficeFile(f)
            office_file.load_key(password=password)
            with tempfile.NamedTemporaryFile(suffix=file_path.suffix, delete=False) as tmp:
                office_file.decrypt(tmp)
                return Path(tmp.name)
    except Exception as e:
        raise Exception(f"Decryption failed for {file_path}: {e}")

def load_and_combine_excels(master_path, password=""):
    all_dataframes = []
    excel_suffixes = ['.xlsx', '.xls', '.xlsb']

    for path in Path(master_path).rglob("*ATO lodgement confirmation*"):
        if path.suffix.lower() not in excel_suffixes:
            continue

        original_path = path  # ✅ Store original full path
        file_path = path

        print(f"📄 Found: {original_path}")

        if is_encrypted(file_path):
            if not password:
                print(f"🔒 Skipping encrypted file (no password): {file_path}")
                continue
            try:
                print(f"🔓 Decrypting: {file_path}")
                file_path = decrypt_excel(file_path, password)
            except Exception as e:
                print(f"❌ {e}")
                continue

        try:
            workbook = pd.ExcelFile(file_path)
        except Exception as e:
            print(f"❌ Failed to open {file_path}: {e}")
            continue

        for sheet in workbook.sheet_names:
            try:
                df = pd.read_excel(workbook, sheet_name=sheet, dtype=str, header=0)#engine='openpyxl',
                df['DTT_FILENAME'] = str(file_path)  # Might be temp path
                df['DTT_FILENAME_ORIGINAL'] = str(original_path)  # ✅ Actual user file
                df['DTT_SHEETNAME'] = sheet
                df['DTT_ID'] = range(1, len(df) + 1)
                all_dataframes.append(df)
                print(f"✅ Loaded {sheet} ({len(df)} rows)")
            except Exception as e:
                print(f"❌ Failed to read sheet '{sheet}' in {file_path}: {e}")

    if not all_dataframes:
        print("⚠️ No data loaded.")
        return pd.DataFrame()

    # Combine all data (optionally align columns)
    combined_df = pd.concat(all_dataframes, ignore_index=True, sort=False)
    print(f"\n🎉 Combined total rows: {len(combined_df)} from {len(all_dataframes)} sheets")
    return combined_df

# 🚀 Example usage
master_path = r''
password = ""  # replace with actual password or leave "" for unencrypted only

final_df = load_and_combine_excels(master_path, password)

# final_df = final_df[final_df['Benefit ID'].str.strip().str.lower() != 'grand total']

# Display a preview
print('🎉 Combined total rows after grand total line removal: ',final_df.shape,'\n')
print("\n🔍 Preview of combined DataFrame:")
final_df.head(3)

## Remove trailing spaces and double quotes

In [ ]:
cols=final_df.select_dtypes(['object']).columns
final_df[cols]=final_df[cols].apply(lambda x: x.str.replace('\n','').str.replace('"','').str.strip())

## Sort

In [ ]:
# Sort final_df by the column 'Title'
sorted_df_1 = final_df.sort_values(by='Title', ascending=True)

# Display a preview of the sorted DataFrame
print("\n🔍 Preview of sorted DataFrame:")
sorted_df_1.head(2)

In [ ]:
# Sort final_df by the column 'Title'
sorted_df_2 = final_df.sort_values(by='Entity Name', ascending=True)

# Display a preview of the sorted DataFrame
print("\n🔍 Preview of sorted DataFrame:")
sorted_df_2.head(2)

In [ ]:
# Sort final_df by the column 'Title'
sorted_df_3 = final_df.sort_values(by='EntityName', ascending=True)

# Display a preview of the sorted DataFrame
print("\n🔍 Preview of sorted DataFrame:")
sorted_df_3.head(2)

In [ ]:
def identify_column_differences():
    mismatched_df = []
    for col1 in sorted_df_1.columns:
        for col2 in sorted_df_2.columns:
            if col1 != col2:
                mismatched = sorted_df_1[~sorted_df_1[col1].isin(sorted_df_2[col2])]
                if not mismatched.empty:
                    mismatched['Source Column'] = col1
                    mismatched['Target Column'] = col2
                    mismatched_df.append(mismatched)
    if mismatched_df:
        print(f"⚠️ Found {len(mismatched_df)} mismatched columns between sorted_df_1 and sorted_df_2")

identify_column_differences() 

In [ ]:
def report_missing_entities():
    mismatched_df = []
    for value in sorted_df_3['EntityName'].unique():
        if value not in sorted_df_2['Entity Name'].values:
            mismatched_df.append(value)
    if mismatched_df:
        print(f"⚠️ Found {len(mismatched_df)} mismatched values in 'Title' not found in 'Entity Name': {mismatched_df}")
    else:
        print("✅ All values in 'Title' exist in 'Entity Name'.")

report_missing_entities()

## Mapping

In [ ]:
import re

def map_columns(df):
    # Extract code and name from filename like "BenefitComparison - 1043 (CNSCV).xlsx"
    def extract_code_name(path):
        match = re.search(r'(\d+)\s*\(([^)]+)\)\.xlsx$', str(path))
        if match:
            return match.group(1), match.group(2)
        return None, None

    df['Reporting Entity Code'], df['Reporting Entity Name'] = zip(*df['DTT_FILENAME_ORIGINAL'].map(extract_code_name))

    # Add constant column
    df['FBT Year'] = 

    # Remote area logic
    # df['Remote area'] = df['Business Amount.1'].fillna('0').apply(
    #     lambda x: 'Non-remote' if str(x).strip() in ['0', '0.0', '', 'nan', '$0.00'] else 'Remote'
    # )

    # 💣 Drop pre-existing base column names to avoid rename duplication
    # df = df.drop(columns=[
    #     col for col in ['Total Taxable Value', 'Gross Taxable Value', 'Business Amount']
    #     if col in df.columns
    # ])

    # Rename columns
    df = df.rename(columns={
        'Benefit ID': 'Benefit ID',
    })

    # Final output ordering
    final_cols = [
        'Benefit ID',
    ]

    return df[final_cols]


# 🧠 Apply the mapping
mapped_df = map_columns(final_df)

# 🔍 Preview
print("\n🎯 Mapped DataFrame:")
print(mapped_df.shape,'\n')
mapped_df.head(2)

## Mapping 2

In [ ]:
# Mapping from raw names to cleaned full names
entity_name_map = {
    'CNSCV': 'CNSCV',
    'CNSCE': 'CNSCE Pty Ltd',

}

# Apply mapping
mapped_df['Reporting Entity Name'] = mapped_df['Reporting Entity Name'].str.strip().map(entity_name_map).fillna(mapped_df['Reporting Entity Name'])

mapped_df.tail(2)

## Export

In [ ]:
# Define local file path
local_output_file = r""

# Save the DataFrame to an Excel file locally
mapped_df.to_excel(local_output_file, index=False, sheet_name="")

print(f"File saved at: {local_output_file}")

=========================================================================================================================

# Upload data

In [ ]:
# import pyodbc
# from datetime import datetime
# from sqlalchemy import create_engine

# # Server and database configuration
# server = ''
# database = ''
# connection_string = f"mssql+pyodbc://@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"

# # Create SQL Alchemy engine
# engine = create_engine(connection_string, fast_executemany=True)

# def upload_dataframe_to_sql(df, table_name, initials):
#     # Define the full table name with date and initials
#     today = datetime.now().strftime('%Y%m%d')
#     full_table_name = f'RAW_{table_name}_{today}_{initials}'

#     # Upload the DataFrame to SQL Server
#     df.to_sql(full_table_name, engine, if_exists='replace', index=False, chunksize=10000)

#     # Return the number of rows uploaded and the full table name
#     return len(df), full_table_name

# # Table name mapping
# table_name_mapping = {
#     0: 'SPOTLESS_CONCUR',
# #    1: 'SPOTLESS_GL_450580',
#     # Add mappings as needed
#     # i: 'TableName' i is index of each dataframe
# }

# initials = "EL"  # Replace with the desired initials

# # Specify the index from which to start uploading
# start_from_index = 0  # Change this to start from a different DataFrame
# end_from_index = 0   # Change this to end at a different DataFrame

# # Loop through trimmed_all_dfs and upload each DataFrame within the specified range
# for i, df in enumerate(trimmed_all_dfs):
#     base_table_name = table_name_mapping.get(i, f'{i+1}')  # Default name if not in mapping
#     row_count, full_table_name = upload_dataframe_to_sql(df, base_table_name, initials)
#     print(f"Uploaded {row_count} rows to table {full_table_name}")

In [ ]:
# import pyodbc
# from datetime import datetime
# from sqlalchemy import create_engine

# # Server and database configuration
# server = ''
# database = ''
# connection_string = f"mssql+pyodbc://@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"

# # Create SQL Alchemy engine
# engine = create_engine(connection_string, fast_executemany=True)

# def upload_dataframe_to_sql(df, table_name, initials):
#     # Check for None or duplicate column names
#     if df.columns.hasnans or df.columns.duplicated().any():
#         raise ValueError("DataFrame contains columns with None or duplicate names.")

#     # Define the full table name with date and initials
#     today = datetime.now().strftime('%Y%m%d')
#     full_table_name = f'RAW_{table_name}_{today}_{initials}'

#     # Upload the DataFrame to SQL Server
#     df.to_sql(full_table_name, engine, if_exists='replace', index=False, chunksize=10000)

#     # Return the number of rows uploaded and the full table name
#     return len(df), full_table_name

# initials = "EL"  # Replace with the desired initials

# # Define the base table name
# base_table_name = 'Hierarchy_Changes'  # Replace with your desired base table name or use a mapped name

# # Upload the DataFrame to SQL Server
# if df is not None:
#     try:
#         row_count, full_table_name = upload_dataframe_to_sql(df, base_table_name, initials)
#         print(f"Uploaded {row_count} rows to table {full_table_name}")
#     except ValueError as e:
#         print(f"Error: {e}")
# else:
#     print("The DataFrame is None and was not uploaded.")


# Read excel
- Inconsistent format

##    Part 1:

        - Search for Excel files in all subfolders of the specified master_path
        - Categorize these files by their extension (xlsx, xls, xlsb)
        - Print the count and the paths of the files in each category
        - Return a dictionary where each key is a file extension and the value is a list of paths to files with that extension

In [ ]:
# from pathlib import Path
# from glob import glob


# def detect_excel_files(master_path):
#     excel_files = {'xlsx': [], 'xls': [], 'xlsb': []}

#     for file_path in master_path.rglob('*.*'):
#         if file_path.suffix.lower() in ['.xlsx', '.xls', '.xlsb']:
#             excel_files[file_path.suffix.lower()[1:]].append(file_path)

#     for key in excel_files:
#         print(f"{len(excel_files[key])}.{key} files found:")
#         for file in excel_files[key]:
#             print(f"  - {file}")

#     return excel_files

# # Define the master path
# master_path=Path(r'C:\Users\ele\Downloads\New folder')
# #raw_path=master_path/'1.Original'
# #input_path=master_path/'2.Precleaned'
# #output_path=master_path/'3.Cleaned'
# excel_files = detect_excel_files(master_path)


##    Part 2:

        - is_encrypted checks if a file is encrypted
        - decrypt_excel handles the decryption of an encrypted file
        - check_and_decrypt_files iterates over all detected Excel files, checks if they are encrypted, and decrypts them if necessary
        It prints the status of each file and stores the paths to the decrypted files

In [ ]:
# import msoffcrypto
# import tempfile

# def is_encrypted(file_path):
#     try:
#         with open(file_path, "rb") as file:
#             office_file = msoffcrypto.OfficeFile(file)
#             return office_file.is_encrypted()
#     except Exception as e:
#         print(f"Error checking if file is encrypted: {e}")
#         return False

# def decrypt_excel(file_path, password):
#     with open(file_path, "rb") as file:
#         office_file = msoffcrypto.OfficeFile(file)
#         office_file.load_key(password=password)

#         with tempfile.NamedTemporaryFile(suffix=file_path.suffix, delete=False) as temp_file:
#             office_file.decrypt(temp_file)
#             return Path(temp_file.name)

# def check_encryption_status(excel_files, password=""):
#     encrypted_files = []
#     for file_type in excel_files:
#         for file in excel_files[file_type]:
#             if is_encrypted(file):
#                 encrypted_files.append(file)

#     if not encrypted_files:
#         print("No encrypted files found.")
#     else:
#         for file in encrypted_files:
#             print(f"{file} is encrypted.")

# check_encryption_status(excel_files, "")


    Part 3:

        - If a file is encrypted, it is decrypted before attempting to read it
        - Any issues in reading a workbook (perhaps due to incorrect decryption or file corruption) are caught and reported

In [ ]:
# def analyze_workbooks(excel_files, password=""):
#     workbook_info = []

#     for file_type in excel_files:
#         for file_path in excel_files[file_type]:
#             if is_encrypted(file_path):
#                 if not password:
#                     print(f"⚠️ Skipping encrypted file (no password provided): {file_path}")
#                     continue

#                 print(f"🔓 Attempting to decrypt: {file_path}")
#                 try:
#                     file_path = decrypt_excel(file_path, password)
#                 except Exception as e:
#                     print(f"❌ Decryption failed for {file_path}: {e}")
#                     continue

#             try:
#                 workbook = pd.ExcelFile(file_path)
#                 for sheet_name in workbook.sheet_names:
#                     workbook_info.append({
#                         'WorkbookName': file_path.name,
#                         'WorksheetName': sheet_name,
#                         'PathName': file_path
#                     })
#             except Exception as e:
#                 print(f"❌ Error reading {file_path}: {e}")

#     return workbook_info


    Part 4:

        - all_dfs is a list that will store all unique DataFrames.
        - Each new sheet_df is compared with the DataFrames already in all_dfs.
        - If a match is found, sheet_df is appended to the matching DataFrame.
        - If no match is found, sheet_df is added as a new unique DataFrame to all_dfs.
        - The function returns all_dfs, which contains all consolidated DataFrames

In [ ]:
# def same_columns(df1, df2):
#     """
#     Check if two DataFrames have the same columns (ignoring order).
#     """
#     return set(df1.columns) == set(df2.columns)

# def consolidate_workbooks(workbook_info, password=""):
#     if not workbook_info:
#         print("❌ No workbooks to consolidate.")
#         return []

#     all_dfs = []
#     file_timestamps = {}

#     for info in workbook_info:
#         workbook_path = info['PathName']
#         sheet_name = info['WorksheetName']

#         if is_encrypted(workbook_path):
#             if not password:
#                 print(f"⚠️ Skipping encrypted file in consolidation (no password): {workbook_path}")
#                 continue
#             try:
#                 workbook_path = decrypt_excel(workbook_path, password)
#             except Exception as e:
#                 print(f"❌ Failed to decrypt during consolidation: {workbook_path}: {e}")
#                 continue

#         try:
#             sheet_df = pd.read_excel(workbook_path, sheet_name=sheet_name, header=0, dtype=str, engine='openpyxl')
#         except Exception as e:
#             print(f"❌ Failed to read sheet {sheet_name} in {workbook_path}: {e}")
#             continue

#         sheet_df['DTT_FILENAME'] = str(workbook_path)

#         matched = False
#         for i, df in enumerate(all_dfs):
#             if same_columns(df.iloc[:, 1:], sheet_df.iloc[:, 1:]):  # Ignore index col
#                 start_id = len(df) + 1
#                 sheet_df['DTT_ID'] = range(start_id, start_id + len(sheet_df))
#                 all_dfs[i] = pd.concat([df, sheet_df], ignore_index=True)
#                 matched = True
#                 break

#         if not matched:
#             sheet_df['DTT_ID'] = range(1, len(sheet_df) + 1)
#             all_dfs.append(sheet_df)

#     return all_dfs

# # Prompt for password interactively if needed
# from getpass import getpass

# # Check if any files are encrypted
# password = "FBT2025"
# if any(is_encrypted(f) for ft in excel_files for f in excel_files[ft]):
#     password = getpass("🔑 Enter password to decrypt encrypted files: ")

# # Run safely
# workbook_info = analyze_workbooks(excel_files, password)
# all_dfs = consolidate_workbooks(workbook_info, password)

# # Display results
# for i, df in enumerate(all_dfs):
#     print(f"\nDataFrame {i+1} Preview:")
#     display(df.head(5))
#     print(f"Total rows in DataFrame {i+1}: {len(df)}")


## Remove double quotes and trim trailing space

In [ ]:
# def double_quotes_and_trim_dataframes(dfs):
#     trimmed_dfs = []  # List to store cleaned DataFrames

#     for df in dfs:
#         # Create a copy of the DataFrame
# #        df_copy = df.copy()

#         # Select columns of object type
#         cols = df.select_dtypes(['object']).columns

#         # Apply the string operations only to string values
#         for col in cols:
#             df[col] = df[col].apply(lambda x: x.replace('"', '').strip() if isinstance(x, str) else x)
        
#         trimmed_dfs.append(df)

#     return trimmed_dfs

# # Apply the cleaning process to all DataFrames
# trimmed_all_dfs = double_quotes_and_trim_dataframes(all_dfs)


## Export

In [ ]:
# # Define local file path
# local_output_file = r"C:\Users\ele\Downloads\PMC\trimmed_all_dfs.xlsx"

# # Save the DataFrame to an Excel file locally
# trimmed_all_dfs[0].to_excel(local_output_file, index=False, sheet_name="Processed Data")

# print(f"File saved locally at: {local_output_file}")